In [46]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import pandas as pd
import os, dotenv, json


ENV_PATH = "../.env"
dotenv.load_dotenv(dotenv_path=ENV_PATH)

False

In [47]:
# Load the model
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

## 

In [54]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(name="hackathon24s")

In [73]:
#index,channel,episode,chapter,start_time,end_time,text,file_name_prefix,prev_index,next_index

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


df = pd.read_csv("config/data.csv")
sentences = []
for i in range(len(df)):
    if i == 128:
        continue
    row = df.iloc[i]
    sentence = row["text"]  
    sentences.append(sentence)

# Load AutoModel from huggingface model repository
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
# model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").to(device)
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-l").to(device)
def generate_embedding(sentence):
    # encoded_input = tokenizer(
    #     sentence, padding=True, truncation=True, max_length=128, return_tensors="pt"
    # ).to(device)
    with torch.no_grad():
        model_output = model.encode(sentence, convert_to_tensor=True)
    return model_output

You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [79]:
## load "config/data.csv" as a DataFrame
#index,channel,episode,chapter,start_time,end_time,text,file_name_prefix,prev_index,next_index

# for each line in csv
#     extract text to embe
#     perform embedding and get vector
#     create vectors 
#     add to upsert list 

upsert_list = []
for i in range(len(df)):
    if i == 128:
        continue
    row = df.iloc[i]
    text = row["text"]
    ## run model encode on gpu
    embedding = generate_embedding(text)
    id = row["index"]
    ## get type of first item in embedding
    embedding = embedding.cpu().numpy().flatten()
    metadata = {"index": str(row["index"]), "channel": str(row["channel"]), "episode": str(row["episode"]), "chapter": str(row["chapter"]), "start_time": str(row["start_time"]), "end_time": str(row["end_time"]), "text": str(row["text"]), "file_name_prefix": str(row["file_name_prefix"]), "prev_index": str(row["prev_index"]), "next_index": str(row["next_index"])}
    vector = ({"values": embedding, "id": str(id), "metadata": metadata})
    upsert_list.append(vector)

In [75]:
print(len(upsert_list))
print(upsert_list[0])

193
{'values': array([ 0.01859549, -0.04389406,  0.03972551, ..., -0.03461152,
        0.01294052, -0.0280786 ], dtype=float32), 'id': '1', 'metadata': {'index': '1', 'channel': 'a16z', 'episode': 'A Nuclear Comeback: Are New Reactors the Answer?', 'chapter': 'The Promise of Advanced Nuclear Reactors', 'start_time': '0.0', 'end_time': '197.0', 'text': "they recognize we have this insatable need for energy like we're not going backwards in our energy consumption so if we're going to have new energy generation has to be clean energy deliveries of fuel are a clear vulnerability natural gas obviously can hold an entire nation hostage the typical construction timeline is really like 6 to 15 years on the big reactors right now the really exciting thing for me is that at the really far into the scale the portable micro reactors we haven't really achieved that yet you could actually produce these in a factory because they're portable you could do mass production maybe nuclear energy is a lot s

In [78]:
print(type(index))

index.upsert(upsert_list, batch_size=10)

<class 'pinecone.data.index.Index'>


Upserted vectors:   0%|          | 0/193 [00:00<?, ?it/s]

{'upserted_count': 193}